In [1]:
import streamlit as st
import os
import time
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from crewai_tools import tool

from docx import Document
from io import BytesIO
import base64

import pandas as pd
from IPython.display import Markdown, display

from dotenv import load_dotenv
#load_dotenv("./.env")
load_dotenv("/home/jovyan/.env")

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
os.environ["SLIP_AUTH_URL"] = os.environ.get("SLIP_AUTH_URL")
os.environ["SLIP_USERNAME"] =os.environ.get("SLIP_USERNAME")
os.environ["SLIP_PASSWORD"] =  os.environ.get("SLIP_PASSWORD")
os.environ["SLIP_CLIENT_ID"]= os.environ.get("SLIP_CLIENT_ID")
os.environ["SLIP_APP_NAME"]=os.environ.get("SLIP_APP_NAME")
os.environ["SERPER_API_KEY"] = "25d43901c968b0f8830387b7ed72231673428b9c"

In [2]:
from llm_utils import get_llm
llm = get_llm('gpt-4o-0513')

In [3]:
df=pd.read_csv('./diag_5mcids.csv')
df

,mcid,dos,codeIndex,cd_value,ICD_FULL_DESC,type
0,0,2017-02-28,13415,M25551,Pain in right hip,diag
1,0,2017-06-27,13415,M25551,Pain in right hip,diag
2,0,2017-08-22,13415,M25551,Pain in right hip,diag
3,0,2018-02-22,13415,M25551,Pain in right hip,diag
4,0,2018-04-18,13415,M25551,Pain in right hip,diag
...,...,...,...,...,...,...
816,4,2018-11-01,14174,M546,Pain in thoracic spine,diag
817,4,2019-01-11,14174,M546,Pain in thoracic spine,diag
818,4,2019-05-07,14174,M546,Pain in thoracic spine,diag
819,4,2018-11-29,9753,J309,"Allergic rhinitis, unspecified",diag


In [4]:
df_json = df.to_json()
import json


In [5]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()


In [6]:
@tool
def patientRecordSelect_tool(df_list, mcid):
    """Select the patient's medical records for patient with given mcid.
    'dos': date of service.
    'cd_value': ICD10 diagnosis codes.
    'ICD_FULL_DESC': description of ICD10 code.
    """
    df = pd.DataFrame(df_list)
    output = df[df['mcid'] == mcid][['dos', 'cd_value', 'ICD_FULL_DESC']]
    return output

patientRecordSelect_agent = Agent(
    role="Get patient's medical records",
    goal="For a given patient identified by mcid, get this patient's medical recordes "
         "from a given data using tool patientRecordSelect_tool.",
    backstory="You use given mcid to find the corresponding medical records.",
    tools=[patientRecordSelect_tool],
    allow_delegation=False,
    llm=llm  #Make sure this variable is defined
)

patientRecordSelect_task = Task(
    description="Get the medical records for the given patient {mcid} from file {df_list}. These medical records will be used in later steps",
    expected_output="a table with columns: date of service, ICD10 diagnosis codes, code description",
    agent=patientRecordSelect_agent
)

summary_agent = Agent(
    role="Summarizer",
    goal="Summarize patient's medical records. Find out what diseases this patient had or having, and possible diseases may happen soon. ",
    tools = [scrape_tool, search_tool],
    verbose=True,
    llm = llm,
    backstory=(
        "As a medical doctor, you have a new patient who brings to you his/her past medical records."
        "Through analyzing this patient's medical records, you find what diseases this patient had in the past, "
        "what chronic desease this patient are having, "
        "and what desease this patient may have in the near future."
    )
)

summary_task = Task(
    description=(
        "1. Analyze the patient's medical records .\n"
        "2. Provide a summary of past, present, and possible future diseases based on the provided information."
        "3. Limit the length to 200 words."
    ),
    expected_output="A summary of patient's medical conditions.",
    output_file="summary.txt",
    agent=summary_agent,
    context=[patientRecordSelect_task]
)

commonDisease_agent = Agent(
    role="Detector of common diseases",
    goal="Find out if the given patient has diabetes and chronic kidney disease.",
    tools = [scrape_tool, search_tool],
    verbose=True,
    llm = llm,
    backstory=(
        "Diabetes, chronic kidney disease are very common, especially among older people. "
        "We are dealing with medicare population. Analyzing a patient's medical records to see if this patient has these diseases. "
        "If this patient has any of the mentioned diseases, please suggest treatment plan, one for each detected disease."
    )
)

commonDisease_task = Task(
    description=(
        "1. Analyze the patient's medical records .\n"
        "2. Answer yes or no for diabetes and chronic kidney disease."
        "3. If the answer is yes, suggest treatment plan and dietary restrictions. When searching the internet, make sure check out Mayo Clinic website."
        "4. Limit the length to 200 words."
        
    ),
    expected_output="Yes or no answer to common chronic diseases. Suggestions if the answer is yes.",
    output_file="commonDisease.txt",
    agent=commonDisease_agent,
    context=[patientRecordSelect_task, summary_task]
)


In [7]:
chatbot_agent = Agent(
    role="Medical Information Chatbot",
    goal="Provide accurate and helpful responses to user questions about the patient's medical condition or general medical queries",
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "You are an AI-powered medical chatbot with the ability to analyze and interpret provided medical records, "
        "summaries, and assessments. You can answer questions "
        "specific to a patient as well as general medical queries. You have access to the outputs of "
        "previous analyses including patient record selection, medical summary, and common disease assessment. "
        "You can also search for additional information when needed to provide comprehensive answers."
    )
)

In [8]:
chatbot_task = Task(
    description=(
        "As a medical chatbot, your task is to answer '{user_question}' asked by the user accurately and helpfully. Follow these steps:\n"
        "1. Review the following patient-specific information:\n"
        "   a. Patient record selection results\n"
        "   b. Medical summary\n"
        "   c. Common disease assessment\n"
        "2. For the user question:\n"
        "   a. First, attempt to answer using the patient-specific information provided.\n"
        "   b. If the patient-specific information is insufficient, then use your general medical knowledge or perform a web search.\n"
        "   c. Provide a clear, concise, and accurate answer, prioritizing patient-specific information when available.\n"
        "   d. Ensure your response is easy for a non-medical professional to understand.\n"
        "   e. If the question is outside your scope or requires immediate medical attention, advise the user to consult a healthcare professional.\n"
        "   f. If you need to make any assumptions or if there's any ambiguity, clearly state this in your response."
    ),
    expected_output="A clear, informative, and accurate response to the user's question, prioritizing patient-specific information when available.",
    agent=chatbot_agent,
    output_file="response.txt",
    context=[patientRecordSelect_task, summary_task, commonDisease_task]
)

In [9]:
# Create Crew including all agents
crew = Crew(
    agents=[patientRecordSelect_agent, summary_agent, commonDisease_agent, chatbot_agent],
    tasks=[patientRecordSelect_task, summary_task, commonDisease_task, chatbot_task],
    verbose=1
)# Create Crew for initial analysis
analysis_crew = Crew(
    agents=[patientRecordSelect_agent, summary_agent, commonDisease_agent],
    tasks=[patientRecordSelect_task, summary_task, commonDisease_task],
    verbose=1
)

# Create separate Crew for chatbot
chatbot_crew = Crew(
    agents=[chatbot_agent],
    tasks=[chatbot_task],
    verbose=1
)

2024-08-01 07:09:50,544 - 139634076358464 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed
2024-08-01 07:09:50,547 - 139634076358464 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [10]:
def main():
   
    # Convert the JSON string back to a dictionary of dictionaries
    df_list = json.loads(df_json)
    
    mcid = input("Enter Patient MCID: ")

    # Run the initial analysis
    initial_results = analysis_crew.kickoff({"df_list": df_list, "mcid": mcid})

    print("\nInitial Analysis Complete. You can now ask questions about the patient.")

    # Chatbot interaction
    while True:
        user_question = input("\nAsk a question related to the patient records or any medical query (or type 'exit' to quit): ")
        if user_question.lower() == 'exit':
            break
            
        # Run the chatbot task with the user's question
        chatbot_response = chatbot_crew.kickoff({"user_question": user_question})[-1]  # Assuming chatbot_task is the last task
        print("Chatbot:", chatbot_response)
    
    return initial_results,chatbot_response

In [11]:
a,b=main()

Enter Patient MCID:  2


 [2024-08-01 07:10:51][DEBUG]: == Working Agent: Get patient's medical records
 

I encountered an error while trying to use the tool. This was the error: patientRecordSelect_tool() missing 1 required positional argument: 'df_list'.
 Tool patientRecordSelect_tool accepts these inputs: patientRecordSelect_tool() - Select the patient's medical records for patient with given mcid.     'dos': date of service.     'cd_value': ICD10 diagnosis codes.     'ICD_FULL_DESC': description of ICD10 code.     

Formatting errors incremented
 [2024-08-01 07:17:15][DEBUG]: == [Get patient's medical records] Task output: | date of service | ICD10 diagnosis codes | code description                    |
|-----------------|-----------------------|-------------------------------------|
| 2017-06-07      | E785                  | Hyperlipidemia, unspecified         |
| 2019-05-08      | E785                  | Hyperlipidemia, unspecified         |
| 2019-05-09      | E785                  | Hyperlipidemia, u


Ask a question related to the patient records or any medical query (or type 'exit' to quit):  does patient has diabetes


 [2024-08-01 07:22:19][DEBUG]: == Working Agent: Medical Information Chatbot


> Entering new CrewAgentExecutor chain...
Based on the patient-specific information provided, I can confirm that the patient has been diagnosed with Type 2 diabetes mellitus. This is explicitly stated in the provided context.

Final Answer: Yes, the patient has been diagnosed with Type 2 diabetes mellitus.

> Finished chain.
 [2024-08-01 07:22:21][DEBUG]: == [Medical Information Chatbot] Task output: Yes, the patient has been diagnosed with Type 2 diabetes mellitus.


Chatbot: .



Ask a question related to the patient records or any medical query (or type 'exit' to quit):  does the patient has heart disease


 [2024-08-01 07:25:05][DEBUG]: == Working Agent: Medical Information Chatbot


> Entering new CrewAgentExecutor chain...
To determine if the patient has heart disease, I will first review the provided patient-specific information, including the ICD10 diagnosis codes and the medical summary.

Based on the patient-specific information provided:
- The patient has a history of hyperlipidemia (E785), chronic pain (G8929), and essential hypertension (I10).
- The patient also has conditions related to the spine and back, such as spondylosis with radiculopathy (M4725), intervertebral disc degeneration (M5135), and low back pain (M545).
- Additionally, the patient has been diagnosed with Type 2 diabetes mellitus and chronic kidney disease.

However, there is no direct mention or diagnosis of heart disease (e.g., ischemic heart disease, myocardial infarction, congestive heart failure) in the provided information.

Thought: Based on the provided patient-specific information, there is no direct di


Ask a question related to the patient records or any medical query (or type 'exit' to quit):  what is nlp


 [2024-08-01 08:16:50][DEBUG]: == Working Agent: Medical Information Chatbot


> Entering new CrewAgentExecutor chain...
Thought: The user's question is "What is NLP?" Given the context and the patient's information provided, it's clear that the question is not specific to the patient's medical conditions. Therefore, I will use general medical knowledge or perform a web search to provide an accurate answer to what "NLP" stands for in a medical context.

Action: Search the internet
Action Input: {"search_query": "What is NLP in medical terms?"}
 


Search results: Title: Natural Language Processing of Clinical Notes on Chronic Diseases
Link: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6528438/
Snippet: Consequently, unlocking the full potential of EHR data is contingent on the development of natural language processing (NLP) methods to automatically transform ...
---
Title: 6 Uses for Natural Language Processing in Healthcare
Link: https://global.hitachi-solutions.com/blog/nlp-in-healt


Ask a question related to the patient records or any medical query (or type 'exit' to quit):  exit
